In [ ]:
#Used to upload files to the notebook. Needs to reupload everyday. 
from google.colab import files
uploaded = files.upload()

In [3]:
import pandas as pd
import numpy as npn
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [ ]:
features = np.load("histograms.npy")
targets = np.load("labels.npy")

In [ ]:
features = np.array(features, dtype=np.float32)
targets = np.array(targets, dtype=np.float32)

In [ ]:
features_train, features_test, targets_train, targets_test = train_test_split(
      features, targets, test_size = 0.3, random_state = 100)
features_train = torch.from_numpy(features_train)
features_test = torch.from_numpy(features_test)
targets_train = torch.from_numpy(targets_train)
targets_test = torch.from_numpy(targets_test)

In [4]:
class Perceptron(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Perceptron, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, 2)
    def forward(self, x):
        x = F.sigmoid(self.layer1(x))
        x = F.sigmoid(self.layer2(x))
        return x

In [ ]:
def eucledian_distance(estimate, target):
    return ((estimate - target) ** 2).sum(axis = 0)

In [ ]:
def train_network(network, epoch, features, labels, optimizer, log_interval,
                  train_losses, train_counter):
    network.train()
    for i in range(len(features)):
        optimizer.zero_grad()
        output = network(features[i])
        loss = eucledian_distance(output, labels[i])
        loss.backward()
        optimizer.step()
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i, len(features),
                       100. * i / len(features), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                i + ((epoch - 1) * len(features)))
            torch.save(network.state_dict(), 'perceptron_model.pth')
            torch.save(optimizer.state_dict(), 'perceptron_optimizer.pth')

In [ ]:
def test(network, features, labels, test_losses):
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i in range(len(features)):
            output = network(features[i])
            test_loss += eucledian_distance(output, labels[i])
            pred = output
            correct += pred.eq(labels[i].data.view_as(pred)).sum()
            if i % 100 == 0:
                print("\nEstimated: ",output," Expected: ", labels[i])
    test_loss /= len(features)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}\n'.format(
        test_loss))

In [ ]:
def train_network_MSE(network, epoch, features, labels, optimizer, log_interval,
                  train_losses, train_counter):
    network.train()
    for i in range(len(features)):
        optimizer.zero_grad()
        output = network(features[i])
        loss = loss_func(output, labels[i])
        loss.backward()
        optimizer.step()
        if i % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i, len(features),
                       100. * i / len(features), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                i + ((epoch - 1) * len(features)))
            torch.save(network.state_dict(), 'perceptron_model.pth')
            torch.save(optimizer.state_dict(), 'perceptron_optimizer.pth')

In [ ]:
loss_func = torch.nn.MSELoss()
def test_MSE(network, features, labels, test_losses): 
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i in range(len(features)):
            output = network(features[i])
            test_loss += loss_func(output, labels[i])
            pred = output
            correct += pred.eq(labels[i].data.view_as(pred)).sum()
            if i % 100 == 0:
                print("\nEstimated: ",output," Expected: ", labels[i])
    test_loss /= len(features)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}\n'.format(
        test_loss))

In [ ]:
# Hyperparameters for training
n_epochs = 10
learning_rate = 0.01
momentum = 0.5
log_interval = 100
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
network = Perceptron(64, 40)
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate,
                          momentum=momentum)


In [ ]:
# Train and test with Eucledian Distance
train_losses = []
train_counter = []
test_losses = []
test_counter = [i * len(features_train) for i in range(n_epochs + 1)]

test(network, features_test, targets_test, test_losses)
for epoch in range(1, n_epochs + 1):
    train_network(network, epoch, features_train, targets_train, optimizer, log_interval,
                  train_losses, train_counter)
    test(network, features_test, targets_test, test_losses)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Estimated:  tensor([0.5569, 0.5191])  Expected:  tensor([0.3055, 0.4719])

Estimated:  tensor([0.5549, 0.5203])  Expected:  tensor([0.4722, 0.4490])

Estimated:  tensor([0.5539, 0.5258])  Expected:  tensor([0.2092, 0.4658])

Estimated:  tensor([0.5611, 0.5256])  Expected:  tensor([0.2011, 0.4691])

Estimated:  tensor([0.5633, 0.5292])  Expected:  tensor([0.2701, 0.4933])

Estimated:  tensor([0.5558, 0.5326])  Expected:  tensor([0.1963, 0.4695])

Test set: Avg. loss: 0.1120

Train Epoch: 1 [0/1240 (0%)]	Loss: 0.136026
Train Epoch: 1 [100/1240 (8%)]	Loss: 0.003694
Train Epoch: 1 [200/1240 (16%)]	Loss: 0.000807
Train Epoch: 1 [300/1240 (24%)]	Loss: 0.000095
Train Epoch: 1 [400/1240 (32%)]	Loss: 0.002009
Train Epoch: 1 [500/1240 (40%)]	Loss: 0.000499
Train Epoch: 1 [600/1240 (48%)]	Loss: 0.000700
Train Epoch: 1 [700/1240 (56%)]	Loss: 0.000785
Train Epoch: 1 [800/1240 (65%)]	Loss: 0.000850
Train Epoch: 1 [900/1240 (73%)]	Loss: 0.000140
Train Epoch: 1 [1000/1240 (81%)]	Loss: 0.000340
Train 

In [ ]:
# Train and test with MSE
train_losses = []
train_counter = []
test_losses = []
test_counter = [i * len(features_train) for i in range(n_epochs + 1)]

test_MSE(network, features_test, targets_test, test_losses)
for epoch in range(1, n_epochs + 1):
    train_network_MSE(network, epoch, features_train, targets_train, optimizer, log_interval,
                  train_losses, train_counter)
    test_MSE(network, features_test, targets_test, test_losses)


Estimated:  tensor([0.3883, 0.4596])  Expected:  tensor([0.3055, 0.4719])

Estimated:  tensor([0.3908, 0.4551])  Expected:  tensor([0.4722, 0.4490])

Estimated:  tensor([0.1998, 0.4659])  Expected:  tensor([0.2092, 0.4658])

Estimated:  tensor([0.2406, 0.4679])  Expected:  tensor([0.2011, 0.4691])

Estimated:  tensor([0.2899, 0.4730])  Expected:  tensor([0.2701, 0.4933])

Estimated:  tensor([0.2087, 0.4705])  Expected:  tensor([0.1963, 0.4695])

Test set: Avg. loss: 0.0009

Train Epoch: 1 [0/1240 (0%)]	Loss: 0.002506
Train Epoch: 1 [100/1240 (8%)]	Loss: 0.000362
Train Epoch: 1 [200/1240 (16%)]	Loss: 0.000043
Train Epoch: 1 [300/1240 (24%)]	Loss: 0.000004
Train Epoch: 1 [400/1240 (32%)]	Loss: 0.000085
Train Epoch: 1 [500/1240 (40%)]	Loss: 0.000122
Train Epoch: 1 [600/1240 (48%)]	Loss: 0.000175
Train Epoch: 1 [700/1240 (56%)]	Loss: 0.001199
Train Epoch: 1 [800/1240 (65%)]	Loss: 0.000127
Train Epoch: 1 [900/1240 (73%)]	Loss: 0.000007
Train Epoch: 1 [1000/1240 (81%)]	Loss: 0.000291
Train 

In [5]:
# Use the perceptron network to predict illuminatino chromaticity
model = Perceptron(64, 40)
model.load_state_dict(torch.load('perceptron_model.pth'))
model.eval()

Perceptron(
  (layer1): Linear(in_features=64, out_features=40, bias=True)
  (layer2): Linear(in_features=40, out_features=2, bias=True)
)

In [6]:
# Get histogram based on images
import cv2
import pandas as pd
import numpy as np
cam2rgb = np.array([
        1.8795, -1.0326, 0.1531,
        -0.2198, 1.7153, -0.4955,
        0.0069, -0.5150, 1.5081,]).reshape((3, 3))
def linearize(img, black_lvl=2048, saturation_lvl=2**14-1):
    """
    remove black level and saturation according to paper
    """
    return np.clip((img - black_lvl)/(saturation_lvl - black_lvl), 0, 1)


def adjust(cam, img, ilum):
    """
    adjust the illumination to cannonical ground truth
    gt[gt["image"]==img.split(".")[0]].to_numpy().flatten()[1:]
    """
    return np.clip(cam/illum, 0, 1)

    
def to_rgb(cam):
    """
    https://github.com/Visillect/CubePlusPlus/blob/master/challenge/make_preview.py
    """
    cam = np.dot(cam, cam2rgb.T)
    return np.clip(cam, 0, 1)**(1/2.2)


def denormalize(cam):
    """
    scale from 0-1 range to 0-255 range
    """
    return (cam*255).astype(np.uint8)


def binarize(cam, steps=64):
    """
    convert a normalized rgb image to its binarized histogram
    according to the papers sampling method
    """
    #flatten rows and columns
    cam = cam.reshape(cam.shape[0]*cam.shape[1], -1)
    #list for tracking rg chromaticity coordinates
    rg_coords = []
    #iterate over pixels and get coordinates
    for pix_idx in range(cam.shape[0]):
        r = cam[pix_idx][0]
        g = cam[pix_idx][1]
        rg_coords.append((r,g))
    #convert from list to dataframe
    rg_df = pd.DataFrame(rg_coords, columns=["r", "g"])
    #bin coordinates according to step size
    rg_df = rg_df.assign(
        r_cut=pd.cut(rg_df.r, np.sqrt(steps).astype(int), labels=list(range(0,np.sqrt(steps).astype(int)))),
        g_cut=pd.cut(rg_df.g, np.sqrt(steps).astype(int), labels=list(range(0,np.sqrt(steps).astype(int))))
    )
    #zip and reduce to set
    step_coords = list(zip(rg_df.r_cut, rg_df.g_cut))
    step_coords = set(step_coords)
    #initialize grid of zeros
    hist = np.zeros((np.sqrt(steps).astype(int), np.sqrt(steps).astype(int)))
    #turn on signal for present coordiniates
    for x in range(np.sqrt(steps).astype(int)):
        for y in range(np.sqrt(steps).astype(int)):
            if (x,y) in step_coords:
                hist[x][y]=1  
    return hist.flatten()

def make_histogram(img_pth):
    """
    iterate over all images and save binarized histograms
    into a pandas dataframe
    """
    #read image
    cam = cv2.imread(img_pth, cv2.IMREAD_UNCHANGED)
    #bgr -> rgb
    cam = cv2.cvtColor(cam, cv2.COLOR_BGR2RGB)
    #uint16 -> float64
    cam = cam.astype(np.float32)
    #black level and saturation
    cam = linearize(cam)
    #not sure what this step means
    cam = to_rgb(cam)
    hist = binarize(cam, steps=64)
    return hist

In [75]:
hist = make_histogram('00_0125.png')
hist = torch.from_numpy(hist)
hist = hist.type(torch.FloatTensor)

In [76]:
output_two = model(hist)

In [77]:
print(output_two)

tensor([0.2163, 0.4739], grad_fn=<SigmoidBackward0>)


In [78]:
from math import sqrt
r = output_two[0].item()
g = output_two[1].item()
b = 1 - r - g

print(r, g, b)


0.21630540490150452 0.4738996624946594 0.30979493260383606


In [82]:
# Use chromaticity to derive the image under neutral light. 
import cv2
img = cv2.imread("test_00_0125.jpg")
max_r = 0
max_g = 0
max_b = 0
total = 0
total_transformed = 0
for i in range (img.shape[0]):
  for j in range (img.shape[1]):
    total += (img[i, j, 0] + img[i, j, 1] + img[i, j, 2])
    total_transformed += (img[i, j, 0] * b + img[i, j, 1] * g + img[i, j, 2] * r)
addition = (total - total_transformed) / (img.shape[0] * img.shape[1])
for i in range (img.shape[0]):
  for j in range (img.shape[1]):
    total = img[i, j, 0] + img[i, j, 1] + img[i, j, 2]
    total_transformed = img[i, j, 0] * b + img[i, j, 1] * g + img[i, j, 2] * r
    difference = total - total_transformed
    img[i, j, 0] = min(img[i, j, 0] * b + addition * b, 255) 
    img[i, j, 1] = min(img[i, j, 1]* g + addition * g, 255)
    img[i, j, 2] = min(img[i, j, 2] * r + addition * r, 255)





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in ubyte_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in ubyte_scalars
  from ipykernel import kernelapp as app


In [83]:
cv2.imwrite('test_transformed.png', img)

True